**L'objectif principal de ce tutoriel est de mettre en évidence l'importance des étapes de pré-traitement   (pre-processing) que nous devons suivre pour toute analyse de données. Parfois, nous avons tendance à oublier la syntaxe du programme python pour le pré-traitement. L'ensemble de données est utilisé selon ma convenance pour afficher la sortie souhaitée. J'espère que ce tutoriel sera une référence rapide pour tout le monde**

In [ ]:
from pandas import *
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib notebook
from scipy.stats import mode

import os
print(os.listdir("../input"))

In [ ]:
diabetes = pd.read_csv('../input/pima-diabetes/diabetes.csv')

In [ ]:
diabetes.head(5)

### Exploration de données

#### Vérifiez si les types de données sont conformes aux attentes

In [ ]:
diabetes.dtypes

### Optimisations de la mémoire

In [ ]:
memory = diabetes.memory_usage()
print(memory)
print("Total Memory Usage = ",sum(memory))

In [ ]:
diabetes.iloc[:,0:9] = diabetes.iloc[:,0:9].astype('float16')

In [ ]:
memory = diabetes.memory_usage()
print(memory)
print("Total Memory Usage = ",sum(memory))

## Vérifier les statistiques récapitulatives

In [ ]:
diabetes.describe()

In [ ]:
diabetes["Outcome"].value_counts()

## Vérifier les valeurs aberrantes

In [ ]:
fig, axs = plt.subplots()
sns.boxplot(data=diabetes,orient='h',palette="Set2")
plt.show()

### Traiter les valeurs aberrantes

In [ ]:
q75, q25 = np.percentile(diabetes["Insulin"], [75 ,25])
iqr = q75-q25
print("IQR",iqr)
whisker = q75 + (1.5*iqr)
print("Upper whisker",whisker)

In [ ]:
diabetes["Insulin"] = diabetes["Insulin"].clip(upper=whisker)

In [ ]:
fig, axs = plt.subplots()
sns.boxplot(data=diabetes,orient='h',palette="Set2")
plt.show()

### Vérifier les valeurs manquantes

In [ ]:
diabetes.head()

In [ ]:
diabetes.columns

In [ ]:

print((diabetes[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']] == 0).sum())

In [ ]:
diabetes.loc[:,['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI']] = diabetes[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI']].replace(0, np.NaN)
diabetes.head()

In [ ]:
diabetes.isnull().sum()

## Gérer les valeurs manquantes

## A. Supprimer les lignes ayant NaN

In [ ]:
print("Size before dropping NaN rows",diabetes.shape,"\n")

nan_dropped = diabetes.dropna()

print(nan_dropped.isnull().sum())
print("\nSize after dropping NaN rows",nan_dropped.shape)

## Supprimer des lignes / colonnes contenant plus qu'un certain pourcentage de NaN

In [ ]:
diabetes.isnull().mean()

In [ ]:
print("Size before dropping NaN rows",diabetes.shape,"\n")

col_dropped = diabetes.loc[:, diabetes.isnull().mean() < .4]
row_dropped = diabetes.loc[diabetes.isnull().mean(axis=1) < .4, :]

print(nan_dropped.isnull().sum())
print("\nSize after dropping Columns with rows",col_dropped.shape)
print("Size after dropping Columns with rows",row_dropped.shape)

### Imputation de valeurs manquantes

#### Nous pouvons imputer les valeurs manquantes de plusieurs manières. Voici les 3 façons les plus courantes de faire

#### 1. Une valeur constante considérée comme "normale" dans le domaine
#### 2. Statistiques récapitulatives telles que moyenne, médiane, mode
#### 3. Une valeur estimée par algorithme ou modèle prédictif

### Binning (Groupement des données par classe)

In [ ]:
bins = [0,25,30,35,40,100]

group_names = ['malnutrition', 'Under-Weight', 'Healthy', 'Over-Wight',"Obese"]
diabetes['BMI_Class'] = pd.cut(diabetes['BMI'], bins, labels=group_names)
diabetes.head(10)

In [ ]:
diabetes.dtypes

In [ ]:
#Impute the values:
diabetes['BMI_Class'].fillna((diabetes['BMI_Class']).mode()[0], inplace=True)
diabetes['Insulin'].fillna((diabetes['Insulin']).mean(), inplace=True)
diabetes['Pregnancies'].fillna((diabetes['Pregnancies']).median(), inplace=True)

# #Now check the #missing values again to confirm:
print(diabetes.isnull().sum())

### **Scaling (Mise à l'échelle)**

In [ ]:
vector = np.random.chisquare(1,500)
print("Mean",np.mean(vector))
print("SD",np.std(vector))
print("Range",max(vector)-min(vector))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
range_scaler = MinMaxScaler()
range_scaler.fit(vector.reshape(-1,1))
range_scaled_vector = range_scaler.transform(vector.reshape(-1,1))
print("Mean",np.mean(range_scaled_vector))
print("SD",np.std(range_scaled_vector))
print("Range",max(range_scaled_vector)-min(range_scaled_vector))

### Normalisation

In [ ]:
from sklearn.preprocessing import StandardScaler
standardizer = StandardScaler()
standardizer.fit(vector.reshape(-1,1))
std_scaled_vector = standardizer.transform(vector.reshape(-1,1))
print("Mean",int(np.mean(std_scaled_vector)))
print("SD",int(np.std(std_scaled_vector)))
print("Range",max(std_scaled_vector)-min(std_scaled_vector))

### Dummification

In [ ]:
dummified_data = pd.concat([diabetes.iloc[:,:-1],pd.get_dummies(diabetes['BMI_Class'])],axis=1)
dummified_data.head()

### Reshape

In [ ]:
vector.shape

In [ ]:
row_vector = vector.reshape(-1,1)
row_vector.shape

In [ ]:
col_vector = vector.reshape(1,-1)
col_vector.shape

In [ ]:
matrix = vector.reshape(10,50)
matrix.shape

### Pivot Table

In [ ]:
#Determine pivot table
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                       "bar", "bar", "bar", "bar"],
                    "B": ["one", "one", "one", "two", "two",
                          "one", "one", "two", "two"],
                    "C": ["small", "large", "large", "small",
                          "small", "large", "small", "small",
                          "large"],
                    "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                    "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})

In [ ]:
df.head(3)

In [ ]:
table = pivot_table(df, values='D', index=['A', 'B'],
                  columns=['C'], aggfunc=np.sum)
table


http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html


### Crosstab

In [ ]:
pd.crosstab(df["D"],df["B"],margins=True)

### Fusion de dataframes

In [ ]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                        'B': ['B0', 'B1', 'B2', 'B3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']},
                       index=[0, 1, 2, 3])
    

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                        'B': ['B4', 'B5', 'B6', 'B7'],
                        'C': ['C4', 'C5', 'C6', 'C7'],
                        'D': ['D4', 'D5', 'D6', 'D7']},
                       index=[4, 5, 6, 7])
    

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                       'B': ['B8', 'B9', 'B10', 'B11'],
                        'C': ['C8', 'C9', 'C10', 'C11'],
                        'D': ['D8', 'D9', 'D10', 'D11']},
                       index=[8, 9, 10, 11])
    

frames = [df1, df2, df3]

result = pd.concat(frames)

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

### Plotting

In [ ]:
grade = pd.read_csv("../input/grades/Grade.csv")
grade.head()

In [ ]:
grade.describe()

In [ ]:
fig, axs = plt.subplots()
sns.regplot(x="StudentId",y="OverallPct1",data=grade,scatter=True,fit_reg=False)
plt.show()

In [ ]:
fig, axs = plt.subplots()
sns.boxplot(data=grade.iloc[:,1:-1],orient='h',palette="Set2")
plt.show()

In [ ]:
# Basic Plots
fig, axs = plt.subplots(ncols=3)
sns.distplot(grade['English1'],ax=axs[0])
sns.distplot(grade['Math1'],ax=axs[1])
sns.distplot(grade['Science1'],ax=axs[2])
plt.show()

In [ ]:
# Axis range and granularity are very important
fig, axs = plt.subplots(ncols=3)
sns.distplot(grade['English1'],ax=axs[0],bins=20)
sns.distplot(grade['Math1'],ax=axs[1],bins=20)
sns.distplot(grade['Science1'],ax=axs[2],bins=20)
axs[0].set_xlim(0,100)
axs[1].set_xlim(0,100)
axs[2].set_xlim(0,100)
axs[0].set_ylim(0,0.12)
axs[1].set_ylim(0,0.12)
axs[2].set_ylim(0,0.12)
plt.show()

In [ ]:
#Asthetics
fig, axs = plt.subplots()
sns.distplot(grade['English1'], bins=10, rug=True, rug_kws={"color": "red"},
             kde_kws={"color": "black", "lw": 2, "label": "KDE"},
             hist_kws={"histtype": "step", "lw": 3,"color": "green"})
plt.show()

In [ ]:
fig, axs = plt.subplots()
normal_dist = np.random.randn(1,1000)
normal_plot = sns.distplot(normal_dist)
normal_plot.set(xlabel='Value', ylabel='Frequency')
plt.show()

In [ ]:
fig, axs = plt.subplots()
vector = np.random.chisquare(1,500)
vector_plot = sns.distplot(vector)
vector_plot.set(xlabel='Value', ylabel='Probability')
plt.show()

In [ ]:
iris = pd.read_csv("../input/iris-dataset/Iris.csv")
iris.head()


In [ ]:
fig, axs = plt.subplots(nrows=2)
sns.swarmplot(x="Species", y="SepalLengthCm",ax=axs[0], data=iris)
sns.swarmplot(x="Species", y="PetalLengthCm",ax=axs[1], data=iris)
plt.show();

In [ ]:
plt.figure(1)
plt.figure(figsize = (12,8))
plt.scatter(iris['PetalLengthCm'], iris['PetalWidthCm'], s=np.array(iris.Species == 'Iris-setosa'), marker='^', c='green', linewidths=5)
plt.scatter(iris['PetalLengthCm'], iris['PetalWidthCm'], s=np.array(iris.Species == 'Iris-versicolor'), marker='^', c='orange', linewidths=5)
plt.scatter(iris['PetalLengthCm'], iris['PetalWidthCm'], s=np.array(iris.Species == 'Iris-virginica'), marker='o', c='blue', linewidths=5)
plt.xlabel('PetalLengthCm')
plt.ylabel('PetalWidthCm')
plt.legend(loc = 'upper left', labels = ['Setosa', 'versicolor', 'virginica'])
plt.show();

**Pour d'autres graphiques et analyses de données exploratoires, veuillez vous référer à mon kernel ici.**

https://www.kaggle.com/shravankoninti/starter-code-and-eda-on-iris-species
![](http://)</div><i class="fa fa-lightbulb-o "></i>